In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 198 kB 62.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=013e49a4b30b5f1383151213e4ededbf410340930cf9e185b2174ebb0133531c
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 36.5 MB of archives.
After t

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1YVBuFSzeGRFJvqsMQAR3d8ikmQS0-IZf'
downloaded = drive.CreateFile({'id': id}) 
downloaded.GetContentFile('soc_LiveJournal.txt')

In [ ]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

import itertools

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
lines = sc.textFile("soc_LiveJournal.txt")
lines = lines.map(lambda x: x.split('\t')) #Splits on tab
lines = lines.map(lambda x: (x[0],x[1].split(','))) #Splits on tab  GOOD UP TO HERE

 # Map pairs for friends
#friends = lines.flatMap(lambda data:[((data[0],friend), 0) for friend in data[1]]) #direct friends
mutualFriends = lines.flatMap(lambda data: [(pair, 1) for pair in itertools.permutations(data[1], 2)]) #Pairs map friend to mutual friend NOT original user to mutual friend
#Need to sort out mutual friends who are already friends with user

#total = friends.union(mutualFriends)

# Reduce to get recommendation list for each user
total = mutualFriends.reduceByKey(lambda x,y: x+y)
mutualCount= total.filter(lambda x:x[1] > 0).map(lambda x: (x[0][0], (x[0][1],x[1]))).groupByKey()

In [ ]:
mutualCount = mutualCount.map(lambda x:(x[0],sorted(x[1],key=lambda x:(int(x[0]))))).map(lambda x:(x[0],sorted(x[1],key=lambda x:-(int(x[1])))))
finalCount = mutualCount.collect() #NEEDS TO FILTER MUTUAL FRIENDS WHO ARE ALREADY FRIENDS

In [ ]:
original=lines.collect() #Lists of users and their friends

args=input()
count=0
suggestedFriends=[]

index=0
for i in original:
  if i[0]==args:
    break
  else:
    index+=1
for i in finalCount:
  if i[0]==args:
    for j in i[1]:
      if j[0] in original[index][1]:
        continue
      else:
        if(len(suggestedFriends)>=10):
          break
        suggestedFriends.append(j[0])
    break
print(', '.join(suggestedFriends))

11
0
1
2
3
4
5
6
7
8
9
27552, 7785, 27573, 27574, 27589, 27590, 27600, 27617, 27620, 27667
